In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
import os
import zipfile
import tempfile
import math
import pathlib

In [ ]:
from google.colab import drive
drive.mount ('/drive')

Mounted at /drive


In [ ]:
train = pd.read_csv('/drive/My Drive/sign_mnist_train.csv')
test = pd.read_csv('/drive/My Drive/sign_mnist_test.csv')

In [ ]:
x_train = train.drop(['pixel1'], axis=1).values
y_train = train['label'].values
x_test = test.drop(['pixel1'], axis=1).values
y_test = test['label'].values
x_train = x_train / 255
x_test = x_test / 255

In [ ]:
x_train = np.array(x_train)
x_test = np.array(x_test)
x_train = x_train.reshape(x_train.shape[0],28,28,1)
x_test = x_test.reshape(x_test.shape[0],28,28,1)

In [ ]:
model = keras.Sequential([
        keras.layers.InputLayer(input_shape=(28, 28, 1)),
        keras.layers.Reshape(target_shape=(28, 28, 1)),
        keras.layers.Conv2D(filters=16, kernel_size=(3,3), activation=tf.nn.relu, padding='same'),
        
        keras.layers.Conv2D(filters=32, kernel_size=(6,6), activation=tf.nn.relu, padding='same'),
        
        keras.layers.Conv2D(filters=64, kernel_size=(12,12), activation=tf.nn.relu, padding='same'),

        keras.layers.MaxPooling2D(pool_size=(2,2)),
        keras.layers.Flatten(),
        keras.layers.Dense(1024, activation='relu'),
        keras.layers.Dropout(0.20),
        keras.layers.Dense(25, activation='softmax')                  
])

In [ ]:
model.compile(optimizer='adam',
          loss='sparse_categorical_crossentropy',
          metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=200)

Epoch 1/10
138/138 [==============================] - 21s 28ms/step - loss: 1.1190 - accuracy: 0.6613 - val_loss: 0.4317 - val_accuracy: 0.8641
Epoch 2/10
138/138 [==============================] - 3s 23ms/step - loss: 0.0263 - accuracy: 0.9936 - val_loss: 0.2537 - val_accuracy: 0.9346
Epoch 3/10
138/138 [==============================] - 3s 23ms/step - loss: 0.0059 - accuracy: 0.9983 - val_loss: 0.2437 - val_accuracy: 0.9279
Epoch 4/10
138/138 [==============================] - 3s 23ms/step - loss: 0.0054 - accuracy: 0.9990 - val_loss: 0.2786 - val_accuracy: 0.9363
Epoch 5/10
138/138 [==============================] - 3s 23ms/step - loss: 0.0135 - accuracy: 0.9962 - val_loss: 0.3222 - val_accuracy: 0.9276
Epoch 6/10
138/138 [==============================] - 3s 23ms/step - loss: 0.0027 - accuracy: 0.9993 - val_loss: 0.2886 - val_accuracy: 0.9364
Epoch 7/10
138/138 [==============================] - 3s 24ms/step - loss: 2.2033e-04 - accuracy: 1.0000 - val_loss: 0.2980 - val_accuracy: 0

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        18464     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        294976    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 12544)             0         
_________________________________________________________________
dense (Dense)                (None, 1024)              1

In [ ]:
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=2)
print('\nTest accuracy:', test_acc)

225/225 - 1s - loss: 0.4053 - accuracy: 0.9336

Test accuracy: 0.9336307644844055


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpllissnas/assets


In [ ]:
tflite_models_dir = pathlib.Path("/tmp/mnist_tflite_models/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

In [ ]:
def representative_data_gen():
  for data in tf.data.Dataset.from_tensor_slices((x_test)).batch(1).take(100):
    yield [tf.dtypes.cast(data, tf.float32)]

converter.representative_dataset = representative_data_gen
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.EXPERIMENTAL_TFLITE_BUILTINS_ACTIVATIONS_INT16_WEIGHTS_INT8]

In [ ]:
tflite_16x8_model = converter.convert()
tflite_model_16x8_file = tflite_models_dir/"sign_mnist_quant_int16x8.tflite"
tflite_model_16x8_file.write_bytes(tflite_16x8_model)

INFO:tensorflow:Assets written to: /tmp/tmpd4327arp/assets


INFO:tensorflow:Assets written to: /tmp/tmpd4327arp/assets


13201744

In [ ]:
interpreter_16x8 = tf.lite.Interpreter(model_path=str(tflite_model_16x8_file))
interpreter_16x8.allocate_tensors()

In [ ]:
# A helper function to evaluate the TF Lite model using "test" dataset.
def evaluate_model(interpreter_16x8):
  input_index = interpreter_16x8.get_input_details()[0]["index"]
  output_index = interpreter_16x8.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  total_seen=0
  total_correct=0
  for img, label in zip(x_test,y_test):
    inp=img.reshape((1,28,28,1))
    total_seen +=1
    interpreter_16x8.set_tensor(input_index, np.float32(inp))


    # Run inference.
    interpreter_16x8.invoke()

    predictions= interpreter_16x8.get_tensor(output_index)

    if np.argmax(label) == np.argmax(predictions):
      total_correct +=1
    if total_seen % 1000 == 0:
      print("Accuracy is".
      format(total_seen,float(total_correct)/float(total_seen)))
      return float(total_correct)/float(total_seen)

In [ ]:
print(evaluate_model(interpreter_16x8))

In [ ]:
def get_gzipped_model_size(file):
  # It returns the size of the gzipped model in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [ ]:
print('Saved 16x8 quantised model model to:', tflite_model_16x8_file)
print('Size of 16x8 quant file model: %.2f bytes' % (get_gzipped_model_size(tflite_model_16x8_file)))